In [1]:
!pip install googletrans==3.1.0a0 -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 5.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
from googletrans import Translator
translator = Translator()

In [3]:
df = pd.read_csv('mahasent_train_new_part1.csv')
df.head()

,Unnamed: 0,tweet,label
0,0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे...,-1
1,1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोल...,-1
2,2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिका...,-1
3,3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्...,1
4,4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...,-1


In [4]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
# for backtranslation, convert all the tweets in the df to English
df['english_translated'] = ""

In [6]:
df.head()

,tweet,label,english_translated
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे...,-1,
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोल...,-1,
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिका...,-1,
3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्...,1,
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...,-1,


In [8]:
# preprocessing: 
def remove_non_marathi_in_tweets(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    # keep punctuations during data augmentation
    # i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [13]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_in_tweets(df['tweet'][i])
df = df[df['tweet']!='']

<ipython-input-13-6e71eba8c7d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i] = remove_non_marathi_in_tweets(df['tweet'][i])


In [17]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    df['english_translated'][i] = translator.translate(df['tweet'][i], src="mr", dest="en").text

  0%|          | 0/6056 [00:00<?, ?it/s]<ipython-input-17-69d591e8be0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['english_translated'][i] = translator.translate(df['tweet'][i], src="mr", dest="en").text
100%|██████████| 6056/6056 [14:21<00:00,  7.03it/s]


In [18]:
# once dataframe is ready, then create a new dataframe, containing the 'english_translated' and 'label' columns only:
df_containing_english = df[['english_translated', 'label']]
df_containing_english.head()

,english_translated,label
0,The news of death of veteran journalist Anant ...,-1
1,Disregarding the directions of the Supreme Cou...,-1
2,Uddhav Thackeray has insulted and betrayed the...,-1
3,Our society is very big. There are many elemen...,1
4,We do not agree with Savarkar who says that ra...,-1


In [19]:
df_containing_english['english_translated'][0]

'The news of death of veteran journalist Anant Dixit is sad. For four decades, he made an invaluable contribution to journalism with his skillful penmanship. A generation of journalists was born under the guidance of Dixit. A moving tribute to Anant Dixit! We share the grief of his family.'

In [20]:
df['tweet'][1]

'सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.'

In [21]:
# now, translate the sentences back to Marathi:
translator.translate(df_containing_english['english_translated'][1], src="en", dest="mr").text

'सर्वोच्च न्यायालयाच्या निर्देशांची अवहेलना करून पुणे पोलिसांनी प्रा.आनंद तेलतुंबडे यांना अटक केली, मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला चांगलेच फटकारले. कायदा हातात घेणाऱ्या पोलिस अधिकाऱ्यांवर तातडीने कारवाई करावी.'

In [22]:
df_backtranslated_english = df_containing_english
df_backtranslated_english.head()

,english_translated,label
0,The news of death of veteran journalist Anant ...,-1
1,Disregarding the directions of the Supreme Cou...,-1
2,Uddhav Thackeray has insulted and betrayed the...,-1
3,Our society is very big. There are many elemen...,1
4,We do not agree with Savarkar who says that ra...,-1


In [23]:
for i in tqdm(range(len(df_backtranslated_english))):
    df_backtranslated_english['english_translated'][i] = translator.translate(df_backtranslated_english['english_translated'][i], src="en", dest="mr").text

  0%|          | 0/6056 [00:00<?, ?it/s]<ipython-input-23-3a682e511d23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtranslated_english['english_translated'][i] = translator.translate(df_backtranslated_english['english_translated'][i], src="en", dest="mr").text
100%|██████████| 6056/6056 [15:32<00:00,  6.50it/s]


In [24]:
df_backtranslated_english.rename(columns = {'english_translated':'backtranslated'}, inplace = True)

<ipython-input-24-c1d8d77d279b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtranslated_english.rename(columns = {'english_translated':'backtranslated'}, inplace = True)


In [25]:
df_backtranslated_english.rename(columns = {'backtranslated':'tweet'}, inplace = True)

<ipython-input-25-10eb77923559>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtranslated_english.rename(columns = {'backtranslated':'tweet'}, inplace = True)


In [26]:
df.head()

,tweet,label,english_translated
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...,-1,The news of death of veteran journalist Anant ...
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...,-1,Disregarding the directions of the Supreme Cou...
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...,-1,Uddhav Thackeray has insulted and betrayed the...
3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...,1,Our society is very big. There are many elemen...
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...,-1,We do not agree with Savarkar who says that ra...


In [27]:
df.drop(['english_translated'], axis=1, inplace=True)

In [28]:
df.head()

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...,-1
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...,-1
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...,-1
3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...,1
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...,-1


In [29]:
df_backtranslated_english.head()

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाची ब...,-1
1,सर्वोच्च न्यायालयाच्या निर्देशांची अवहेलना करू...,-1
2,भाजपसोबत युती करून उद्धव ठाकरेंनी शिवसैनिकांचा...,-1
3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक सामील आ...,1
4,बलात्काराचा बदला बलात्काराने घ्यावा असे सांगणा...,-1


In [30]:
len(df_backtranslated_english)

6056

In [31]:
df_backtranslated_english.drop_duplicates()

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाची ब...,-1
1,सर्वोच्च न्यायालयाच्या निर्देशांची अवहेलना करू...,-1
2,भाजपसोबत युती करून उद्धव ठाकरेंनी शिवसैनिकांचा...,-1
3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक सामील आ...,1
4,बलात्काराचा बदला बलात्काराने घ्यावा असे सांगणा...,-1
...,...,...
6051,मुंबईतील विधानभवनात काँग्रेस नेत्याने लिहिलेल्...,0
6052,"आपल्या अखंड, विचारातून, समतेचा, विश्वबंधुतेचा ...",1
6053,यासोबतच काजू व्यापाऱ्यांची मागील कालावधीतील व्...,1
6054,सरकारने कर्जमाफी योजना लागू करून जवळपास २ वर्ष...,-1


In [35]:
df_backtranslated_english['tweet'][1]

'सर्वोच्च न्यायालयाच्या निर्देशांची अवहेलना करून पुणे पोलिसांनी प्रा.आनंद तेलतुंबडे यांना अटक केली, मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला चांगलेच फटकारले. कायदा हातात घेणाऱ्या पोलिस अधिकाऱ्यांवर तातडीने कारवाई करावी.'

In [36]:
df['tweet'][1]

'सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.'

In [37]:
df_final = df.append(df_backtranslated_english, ignore_index = True)

<ipython-input-37-6ddf09676330>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(df_backtranslated_english, ignore_index = True)


In [43]:
df_final.to_csv('backtranslated-augmented-without-paraphrasing-mahasent.csv', index=False)
# here, the punctuations are maintained. remove them during training